## UKPN Dashboard Data Report

In [49]:
# Import all required packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import xarray as xr
import numpy as np
from datetime import datetime, time, timedelta
import calendar
import random

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 5

import sys
sys.path.insert(0, "/home/raj/ocf/pv-solar-farm-forecasting/ukpn/scripts")

from resample_data import load_csv_to_pandas, count_total_gsp_solar, interpolation_pandas

%matplotlib inline

In [ ]:
# Total Count - GSP Data

# Plotting a bar graph
folder_destination = "/home/raj/ocf/pv-solar-farm-forecasting/tests/data/ukpn_dashboard_data"
count_dict = count_total_gsp_solar(folder_destination = folder_destination)
bar_labels = list(count_dict.keys())
bar_values = list(count_dict.values())

# plotting using seaborn
sns.set_theme(style='darkgrid', rc={'figure.dpi': 147},              
              font_scale=0.7)
fig, ax = plt.subplots(figsize=(7, 2))
ax.set_title(" Total count of the GSP data as of Today (18/01/2023)")
sns.barplot(x = bar_labels, y = bar_values, ax = ax)
ax.set(xlabel = "GSP's", ylabel = "Count")
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [ ]:
# Plotting Canterbury Metered power generation for the current month (18/01/2023)

# Loading the file
path_to_file = "/home/raj/ocf/pv-solar-farm-forecasting/tests/data/ukpn_dashboard_data/canterbury_north.csv"
canterbury_df = load_csv_to_pandas(path_to_file = path_to_file)

# Getting the 2023 Jan data
canterbury_df_jan = canterbury_df[canterbury_df.index.year == 2023]
canterbury_df_jan = canterbury_df_jan[np.in1d(canterbury_df_jan.index.day.values, [15,16,17])]

# Sunrise and Sunset time
sunrise_time = time(7, 50)
sunrise_time = canterbury_df_jan[np.in1d(canterbury_df_jan.index.time, sunrise_time)].index.values
sunset_time = time(16,20)
sunset_time = canterbury_df_jan[np.in1d(canterbury_df_jan.index.time, sunset_time)].index.values

def legend_without_duplicate_labels(figure):
    """Prevents legend labels from duplicating
    link - https://stackoverflow.com/questions/19385639/duplicate-items-in-legend-in-matplotlib
    """
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    figure.legend(by_label.values(), by_label.keys(), loc='upper right')

# Plotting
sns.lineplot(data = canterbury_df_jan, linewidth = 1.5)
for i in range(len(sunrise_time)):
    plt.axvline(sunrise_time[i], linewidth = 1.5, color = 'orange', linestyle = '--', label = 'sunrise')
    plt.axvline(sunset_time[i], linewidth = 1.5, color = 'red', linestyle = '--', label = 'sunset')
    # Naming legend
    legend_without_duplicate_labels(plt)
    plt.title("Timeseries data of Power Genearion (MW) - Canterbury North")
    plt.xlabel("Date")
    plt.ylabel("Metered Power Generation (MW)")


In [ ]:
# Classification of power generated by seasons

# Loading the file
path_to_file = "/home/raj/ocf/pv-solar-farm-forecasting/tests/data/ukpn_dashboard_data/canterbury_north.csv"
canterbury_df = load_csv_to_pandas(path_to_file = path_to_file)

# Resetting the index
canterbury_df.reset_index(inplace=True)

# Defining Seasons
seasons = {
    1 : "Winter",
    2 : "Winter",
    3 : "Spring",
    4 : "Spring",
    5 : "Spring",
    6 : "Summer",
    7 : "Summer",
    8 : "Summer",
    9 : "Autumn",
    10 : "Autumn",
    11 : "Autumn",
    12 : "Winter"
}
# Getting the data for only 2022
canterbury_df_2022 = canterbury_df[canterbury_df["date_time"].dt.year == 2022].reset_index(drop=True)

# Getting the data for the mid-day 11am to 1pm
canterbury_df_2022 = canterbury_df_2022.set_index("date_time")
canterbury_df_2022  = canterbury_df_2022.between_time('11:00', '13:00')

# Getting the average for a single day
canterbury_avg_2022 = canterbury_df_2022.groupby([
    canterbury_df_2022.index.month, canterbury_df_2022.index.day])["canterbury_north"].mean().to_frame()

# Getting the seasons of the year
canterbury_avg_2022.index.set_names(['month' , 'date'], inplace = True)
canterbury_avg_2022.reset_index(inplace = True)
canterbury_avg_2022["season"] = canterbury_avg_2022["month"].apply(lambda month_number: seasons[month_number])
canterbury_avg_2022["month"] = canterbury_avg_2022["month"].apply(lambda month_number: calendar.month_abbr[month_number])

# Plot the data
# This plot classifies the average power generated on every mid-day (11am - 1pm) of a season day
# For example, the first bar represents the average of 1st day of every month in a season
ax = sns.barplot(data = canterbury_avg_2022, x = 'date', y = 'canterbury_north', hue = "season", ci = None)
ax.set_xlabel('Typical days of a month', fontsize = 10)
ax.set_ylabel('Average Power Generated (MW)', fontsize = 10)
ax.axes.set_title("Mid-day avg power(MW) generated in 2022 - Canterbury North", fontsize = 12)
plt.show


In [52]:
# Plotting the negative values before and after interpolation
def check_for_negative_data(
    original_df: pd.DataFrame,
    replace_with_nan: bool = False
    ):
    """This function helps in indentifying if there are any neagtive values

    Args:
        original_df: Loaded dataframe from the csv file
        replace_with_nan: If truem it replaces negative values with NaN's
    """
    # Check for the negative values
    check_non_negative = (original_df < 0).any()
    if not check_non_negative[0]:
        print(f"The CSV file does contain the negative values {check_for_negative_data}")
    else:
        # Filtering the dataframe which has negative values
        negative_df = original_df.iloc[np.where(original_df[original_df.columns[0]].values < 0.)]        
        if not replace_with_nan:
            # Returns index values where there are negative numbers
            return negative_df.index
        else:     
            # Replacing negative values with NaN's
            original_df.loc[negative_df.index] = np.nan
            # Returns original dataframe with negative values replaced with NaN's
            return original_df


# Loading the file
path_to_file = "/home/raj/ocf/pv-solar-farm-forecasting/tests/data/ukpn_dashboard_data/canterbury_north.csv"
canterbury_df = load_csv_to_pandas(path_to_file = path_to_file)


# Getting the indices of the negative values
canterbury_negative_indices = check_for_negative_data(original_df = canterbury_df)
unique_dates_with_negative = canterbury_negative_indices.map(lambda t: t.date()).unique()

# Selecting random dates
select_dates = random.sample(unique_dates_with_negative.to_list(), 1)

# Getting the required dataframe
final_dataframe = pd.DataFrame([])

for i in range(len(select_dates)):
    # Filtering dataframe based on each date with negative values
    filter_df = canterbury_df.loc[pd.to_datetime(canterbury_df.index.values).date == select_dates[i]]
    print(filter_df)
    # Interpolating the missing values
    start_date = select_dates[i].strftime("%Y-%m-%d")
    end_date = (datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days = 1)).strftime("%Y-%m-%d")
    interpolated_df = interpolation_pandas(
        original_df = filter_df, 
        start_date = start_date,
        end_date = end_date,
        freq = "10Min",
        drop_last_row = True)
    
    final_dataframe = pd.concat([final_dataframe, interpolated_df])
    print(final_dataframe)


                     canterbury_north
date_time                            
2022-08-20 00:00:00             0.029
2022-08-20 00:10:00             0.029
2022-08-20 00:20:00             0.086
2022-08-20 00:30:00             0.086
2022-08-20 00:40:00             0.086
...                               ...
2022-08-20 23:10:00            -0.059
2022-08-20 23:20:00            -0.063
2022-08-20 23:30:00            -0.067
2022-08-20 23:40:00            -0.071
2022-08-20 23:50:00            -0.075

[144 rows x 1 columns]


TypeError: interpolation_pandas() got an unexpected keyword argument 'drop_last_row'

In [ ]:
test = pd.date_range(start="2017-01-01", end="2017-01-02", freq="10Min")
test

In [ ]:
def first_plot(
    path_to_file: str
    ):
    # Getting the original data frame
    original_df = load_csv_to_pandas(path_to_file=path_to_file)

    # Getting the slopes of every two rows for the entire data frame column
    original_df["slope"] = original_df["test"].rolling(window = 2).apply(lambda x: x[1]-x[0])
    original_df["slope"] = original_df["slope"].fillna(0)

    # Getting the dates of the days with slopes range 
    active_timestamps = original_df[~original_df["slope"].between(-1.5, 1.5)]

    # Getting the most common times of the data with active power
    active_timestamps = active_timestamps.index.time
    print(np.unique(active_timestamps))
    

    # derivative = original_df[original_df.columns[0]].diff() / original_df.index.to_series().diff().dt.total_seconds()
    # original_df["trend"] = derivative.gt(0).map({False : '-1', True : '1'})
    # print(original_df.head(20))
    original_df = original_df.head(144)
    original_df.plot(y="test", use_index=True)
    plt.xlabel("Date Range")
    plt.ylabel("Gnenerated metered power (MW)")
    plt.title("Time series data of power generated")
    plt.show()
